<a href="https://colab.research.google.com/github/WatcharachaiSk/colab/blob/main/Jobs_Detail_To_LPO_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

def process_job_data(file_path, sheet_name, assign_to_list, alert_keywords, deploy_keywords, ussd_keywords, tk_type_alarm_keywords, tk_type_deploy_keywords, sub_type_alarm_keywords):
    # โหลดข้อมูลในชีตที่ต้องการ
    df_report = pd.read_excel(file_path, sheet_name=sheet_name)

    # กรองข้อมูลในคอลัมน์ 'ASSIGN_TO' ที่มีค่าตรงกับ array
    filtered_df = df_report[df_report['ASSIGN_TO'].isin(assign_to_list)].copy()

    # สร้างคอลัมน์ 'GROUP' โดยตรวจสอบว่า alert_keywords มีอยู่ใน 'TITLE' หรือไม่
    group_conditions = [
        filtered_df['TITLE'].str.lower().str.contains('|'.join([keyword.lower() for keyword in alert_keywords]))
    ]
    group_choices = ['Alarm']
    filtered_df.loc[:, 'GROUP'] = np.select(group_conditions, group_choices, default='')

    # กำหนดค่า 'Problem' โดยการกรองตามเงื่อนไข GROUP
    filtered_df.loc[filtered_df['GROUP'] != 'Alarm', 'TITLE'] = filtered_df['TITLE'] + ' | ' + filtered_df['TASK_DESCRIPTIN']

    # แปลงคอลัมน์ DATE ให้เป็น datetime ก่อน
    filtered_df.loc[:, 'DATE'] = pd.to_datetime(filtered_df['JB_ASSIGN_DATE'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
    # แปลงวันที่ให้เป็นรูปแบบใหม่ 'เดือน-วัน-ปี ชั่วโมง:นาที:วินาที'
    filtered_df.loc[:, 'DATE'] = filtered_df['DATE'].dt.strftime('%m-%d-%Y %H:%M:%S')

    # แปลงคอลัมน์ MONTH
    filtered_df.loc[:, 'MONTH'] = pd.to_datetime(filtered_df['JB_ASSIGN_DATE'], format='%d-%m-%Y %H:%M:%S', errors='coerce').dt.month_name()

    # ทำให้ตัวอักษรใน 'MODULE_NAME'
    filtered_df.loc[:, 'MODULE_NAME'] = np.where(
      filtered_df['TITLE'].str.contains('Privilege|PPRCW', case=False, na=False),
      'Privilege',
      filtered_df['SITE_ACCESS_DESC'].apply(lambda x: x.split('-')[1].capitalize() if len(x.split('-')) > 1 and x.split('-')[1] else x.split('-')[0].capitalize())
    )

    # ทำให้ตัวอักษรใน 'ASSIGN_TO'
    filtered_df.loc[:, 'STAFF'] = filtered_df['ASSIGN_TO'].str.split(' ').str[0].str.capitalize()

    # สร้างคอลัมน์ 'TK_GROUP' โดยตรวจสอบว่า alert_keywords หรือ deploy_keywords มีอยู่ใน 'TITLE' หรือไม่
    tk_group_conditions = [
        filtered_df['TITLE'].str.lower().str.contains('|'.join([keyword.lower() for keyword in alert_keywords])),
        filtered_df['TITLE'].str.lower().str.contains('|'.join([keyword.lower() for keyword in deploy_keywords])),
        filtered_df['TITLE'].str.lower().str.contains('|'.join([keyword.lower() for keyword in ussd_keywords])),
    ]
    tk_group_choices = ['Alarm', 'Deploy', 'Use USSD']
    filtered_df.loc[:, 'TK_GROUP'] = np.select(tk_group_conditions, tk_group_choices, default='')

    # สร้างคอลัมน์ 'TK_TYPE' กรณี TK_GROUP Alarm or Deploy
    tk_type_conditions = [
        filtered_df['TK_GROUP'].str.lower().str.contains('|'.join([keyword.lower() for keyword in tk_type_alarm_keywords])),
        filtered_df['TK_GROUP'].str.lower().str.contains('|'.join([keyword.lower() for keyword in tk_type_deploy_keywords]))
    ]
    tk_type_choices = ['Select', 'SA Request']
    filtered_df.loc[:, 'TK_TYPE'] = np.select(tk_type_conditions, tk_type_choices, default='')

    # สร้างคอลัมน์ 'SubType' กรณี TK_GROUP = Alarm
    sub_type_alarm_keywords_pattern = '|'.join([keyword.lower() for keyword in sub_type_alarm_keywords])
    filtered_df.loc[:, 'SubType'] = filtered_df['TK_GROUP'].str.lower().str.contains(sub_type_alarm_keywords_pattern).apply(lambda x: 'Alarm' if x else '')

    # แทนค่าที่เป็น NaN ในคอลัมน์ 'Cause' ด้วย '-'
    filtered_df.loc[:, 'Cause'] = filtered_df['CAUSE'].fillna('-')

    # แทนค่าที่เป็น NaN ในคอลัมน์ 'Ref_Script' ด้วย '-'
    filtered_df.loc[:, 'Ref_Script'] = filtered_df['REF_SCRIPT'].fillna('-')

    # แทนค่าที่เป็น NaN ในคอลัมน์ 'out_reason' ด้วย '-'
    filtered_df.loc[:, 'out_reason'] = filtered_df['DELAY_REASON'].fillna('-')

    # แทนค่าที่เป็น NaN ในคอลัมน์ 'out_reason' ด้วย '-'
    filtered_df.loc[:, 'Priority'] = filtered_df['PRIORITY'].fillna('-')

    # เลือกคอลัมน์ที่ต้องการและเปลี่ยนชื่อ
    filtered_df = filtered_df[['JB_ID', 'DATE', 'MONTH', 'MODULE_NAME', 'TK_GROUP', 'TK_TYPE', 'STAFF', 'SubType', 'TITLE', 'Cause', 'SOLUTION', 'Ref_Script', 'Priority', 'IN_OUT_SLA', 'out_reason']].rename(columns={'JB_ID': 'TK_NO', 'DATE': 'OPEN_DATE', 'TITLE': 'Problem', 'SOLUTION': 'Solution', 'REF_SCRIPT': 'Ref_Script', 'IN_OUT_SLA': 'SLA'})

    # จัดเรียงข้อมูลตาม STAFF และ OPEN_DATE
    filtered_df = filtered_df.sort_values(by=['STAFF', 'OPEN_DATE'])

    return filtered_df

# เรียกใช้ฟังก์ชัน
file_path = '/content/JOB_DETAIL.xlsx'
sheet_name = 'JOB_DETAIL'
assign_to_list = [
    'Treethos Pimjun',
    'Chinnawat Sihabut',
    'Watcharachai Samkhan',
    'Supanut Thongkum',
    'Patcharawat Suwannapal',
    'Kantapong Dechakiatkrai'
]

alert_keywords = ['alarm', 'alerts', '5XX', 'Kql', 'msg', 'PPRCW']
deploy_keywords = ['Deploy', 'Activity']
ussd_keywords = ['USSD']

tk_type_alarm_keywords = ['Alarm']
tk_type_deploy_keywords = ['Deploy']

sub_type_alarm_keywords = ['Alarm']

# Func
filtered_df = process_job_data(file_path, sheet_name, assign_to_list, alert_keywords, deploy_keywords, ussd_keywords, tk_type_alarm_keywords, tk_type_deploy_keywords, sub_type_alarm_keywords)

filtered_df.head()

# Create Excel
output_file_path = '/content/Ticket_summary.xlsx'
filtered_df.to_excel(output_file_path, index=False)
